In [228]:
##### Import general use tools
from collections import defaultdict
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import dill

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib

import plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# with open('kmb.pik', 'rb') as k:
#     kmb = dill.load(k)
# with open('lsa.pik', 'rb') as ll:
#    lsa = dill.load(ll)

In [ ]:
# with open('binned_data.pik', 'rb') as bd:
#    binned_data = dill.load(bd)

### First, retrieve necessary information.

+ Topics
+ Word counts
+ Years
+ Document names

In [ ]:
# Retrieve the top terms for each cluster from the dictionary, 
# with word counts from the vocab. 

# brute force... do this with classes and recursion

km_exclusions = [0,1,2,3,4,5,"labels","vocab","matrix",(1865, 1875),(1955, 1965)]

def get_top_terms_k(kmb, exclusions): # vocab_idx, dd_type
    keys = list(set(kmb.keys()) - set(exclusions))
    top_terms = defaultdict(list)
    for k in keys:
        for i in range(6):
            try:
                top_terms[k].append({i: {term: kmb[k]["vocab"].get(term) for term in kmb[k][i]}})
            except Exception as e:
                print(f"Exception: {e}")
    return top_terms
kmdict = get_top_terms_k(kmb, km_exclusions) # kmdict[(1980, 1990)][2][2].keys()

In [ ]:
# NB: Matrix indices should equal case_dict indices for case names.

lsa_exclusions = [0, 1, 'vocab', (1955, 1965), (1865, 1875)] # where did these come from?
def get_top_terms(lsa, exclusions):
    keys = list(set(lsa.keys()) - set(exclusions))
    top_terms = defaultdict(list)
    for k in keys:
        try:
            top_terms[k].append({term: lsa[k]["vocab"].get(term) for term in lsa[k]["terms"]})
        except Exception as e:
            print(f"Exception: {e}")
    return top_terms
lsat = get_top_terms(lsa, lsa_exclusions)

In [ ]:
# Match the cases back up to the labels in the kmeans clusters

for k in kmb.keys():
    print(f"KEY: {k}\nLABELS: {len(kmb[k]['labels'])}\nBINNED DATA: {len(binned_data[k])}\n\n")

In [ ]:
# Add the topics back in to the binned cases.

def topic_add(kmb, data):
    topic_add = defaultdict(list)
    for k in kmb.keys():
        topic_add[k].append(list(map(list, zip(data[k], kmb[k]["labels"]))))
    return topic_add
topic_and_data = topic_add(kmb, binned_data)

In [ ]:
topic_and_data[(1870, 1880)][0][0]      # each entry is a list of case and topic; 
                                        #[0][0][0] = case info, [0][0][1] = topic 

In [ ]:
# Check which case has which topic

for k in topic_and_data.keys():
    for case in topic_and_data[k][0]:
        doc_topic = case[1]
        print(f"{case[0][0]} has topic {doc_topic} in years {k}")

In [362]:
big_df = pd.DataFrame(columns=["year_1", "year_2", "title", "topic","terms", "term_frequency", "lsa_bow"])

for k in topic_and_data.keys():
    range_df = pd.DataFrame(columns=["year_1", "year_2", "title", "topic","terms", "term_frequency", "lsa_bow"])
    for i in range(len(topic_and_data[k][0])):
        range_df.loc[i] = {"year_1":k[0],"year_2":k[1],"title":topic_and_data[k][0][i][0][0],
                     "topic":topic_and_data[k][0][i][1],"terms":[kmb[k][topic_and_data[k][0][i][1]]],
                     "term_frequency":[kmdict[k][topic_and_data[k][0][i][1]]],"lsa_bow":[lsat[k]]}
    big_df = pd.concat([big_df, range_df], axis=0)

In [363]:
big_df

,year_1,year_2,title,topic,terms,term_frequency,lsa_bow
0,1870,1880,Worthy v. Commissioners,3,"[[authority exercise, jurisdiction, treaty, di...","[{3: {'authority exercise': 2750, 'jurisdictio...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
1,1870,1880,Osborn v. Nicholson,0,"[[slave, jurisdiction, cease, contract, pleadi...","[{0: {'slave': 32132, 'jurisdiction': 18601, '...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
2,1870,1880,Bradwell v. State,2,"[[citizen, property, government, privilege, pe...","[{2: {'citizen': 5000, 'property': 27082, 'gov...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
3,1870,1880,Slaughter-House Cases,2,"[[citizen, property, government, privilege, pe...","[{2: {'citizen': 5000, 'property': 27082, 'gov...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
4,1870,1880,Bartemeyer v. Iowa,2,"[[citizen, property, government, privilege, pe...","[{2: {'citizen': 5000, 'property': 27082, 'gov...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
5,1870,1880,Minor v. Happersett,2,"[[citizen, property, government, privilege, pe...","[{2: {'citizen': 5000, 'property': 27082, 'gov...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
6,1870,1880,Scholey v. Rew,5,"[[tax, tax duty, estate, succession, duty, suc...","[{5: {'tax': 33814, 'tax duty': 33842, 'estate...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
7,1870,1880,Raymond v. Thomas,0,"[[slave, jurisdiction, cease, contract, pleadi...","[{0: {'slave': 32132, 'jurisdiction': 18601, '...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
8,1870,1880,Walker v. Sauvinet,4,"[[trial, jury, suit common, trial jury, proces...","[{4: {'trial': 34970, 'jury': 18737, 'suit com...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."
9,1870,1880,United States v. REESE,2,"[[citizen, property, government, privilege, pe...","[{2: {'citizen': 5000, 'property': 27082, 'gov...","[[{'ship vessel': 31933, 'water': 36753, 'tria..."


In [225]:
# Sanity check: Make sure the DataFrame matches up

for key in lsa.keys():
    print(lsa[key]["terms"])
print("-"*117)
for key in range(6):
    print(f"{key}: {kmb[(1870, 1880)][key]}")

['ship vessel', 'water', 'trial', 'regulate', 'cabin', 'steamer', 'coast', 'vessel', 'commerce', 'passenger']
['expense', 'corporation', 'bank', 'tax', 'race', 'assessment', 'drainage', 'owner', 'vroom', 'land']
['year', 'born', 'nebraska', 'race', 'kill dog', 'collar', 'citizen', 'kill', 'rate', 'dog']
['tax', 'tax deed', 'health', 'commission', 'title', 'notice', 'sell', 'land', 'deed', 'sale']
['hour day', 'value', 'woman', 'overtime', 'valuation', 'board', 'train', 'labor', 'assessment', 'hour']
['public school', 'board', 'language school', 'pupil', 'teach', 'foreign language', 'taught', 'language', 'child', 'school']
['situs', 'stock', 'street', 'interstate commerce', 'oklahoma', 'car', 'commerce', 'interstate', 'assessment', 'bank']
['texas', 'oil', 'value', 'fare', 'natural gas', 'order', 'picket', 'rate', 'commission', 'gas']
['double jeopardy', 'offense', 'double', 'government', 'picket', 'tax', 'property', 'jeopardy', 'prosecution', 'school']
['interstate', 'reck', 'congress'

In [364]:
# Create a pandas DataFrame to use with Plotly

big_df["years"] = list(zip(big_df["year_1"], big_df["year_2"]))

In [241]:
cols = ["title", "topic", "years", "term_frequency"]
terms = big_df[cols]

In [346]:
big_df["lsa_bow"].apply(lambda x: flatten_all(x))

0      {'ship vessel': '31933', 'water': '36753', 'tr...
1      {'ship vessel': '31933', 'water': '36753', 'tr...
2      {'ship vessel': '31933', 'water': '36753', 'tr...
3      {'ship vessel': '31933', 'water': '36753', 'tr...
4      {'ship vessel': '31933', 'water': '36753', 'tr...
5      {'ship vessel': '31933', 'water': '36753', 'tr...
6      {'ship vessel': '31933', 'water': '36753', 'tr...
7      {'ship vessel': '31933', 'water': '36753', 'tr...
8      {'ship vessel': '31933', 'water': '36753', 'tr...
9      {'ship vessel': '31933', 'water': '36753', 'tr...
10     {'ship vessel': '31933', 'water': '36753', 'tr...
11     {'ship vessel': '31933', 'water': '36753', 'tr...
12     {'ship vessel': '31933', 'water': '36753', 'tr...
13     {'ship vessel': '31933', 'water': '36753', 'tr...
14     {'ship vessel': '31933', 'water': '36753', 'tr...
15     {'ship vessel': '31933', 'water': '36753', 'tr...
16     {'ship vessel': '31933', 'water': '36753', 'tr...
17     {'ship vessel': '31933',

In [327]:
def flatten_all(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + str(a)) 
        elif type(x) is list:
            for a in x:
                flatten(a, a)
        else:
            out[str(name)] = str(x)

    flatten(y)
    return out

In [365]:
for col in ["term_frequency", "lsa_bow"]:
    big_df[col] = big_df[col].apply(lambda x: flatten_all(x))

In [367]:
big_df["terms"] = big_df["terms"].apply(lambda x: x[0])

In [368]:
big_df.head()

,year_1,year_2,title,topic,terms,term_frequency,lsa_bow,years
0,1870,1880,Worthy v. Commissioners,3,"[authority exercise, jurisdiction, treaty, dis...","{'authority exercise': '2750', 'jurisdiction':...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
1,1870,1880,Osborn v. Nicholson,0,"[slave, jurisdiction, cease, contract, pleadin...","{'slave': '32132', 'jurisdiction': '18601', 'c...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
2,1870,1880,Bradwell v. State,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
3,1870,1880,Slaughter-House Cases,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
4,1870,1880,Bartemeyer v. Iowa,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"


### Plotly

In [400]:
# pd.DataFrame.from_dict(kmb)
# pd.DataFrame.from_dict(lsa)
# pd.DataFrame.from_dict(topic_and_data)
# pd.DataFrame.from_dict(lsat)

In [446]:
def flat(x):
    out = []
    for item in x:
        if isinstance(item, dict):
            out = flat([x[k] for k in x])
        elif isinstance(item, list):
            out = [flat[e] for e in x]
        else:
            out = x
    return out

flat(kdataframe.iloc[(0,0)])

{0: {'death': 280198,
  'stay': 999911,
  'execution': 398643,
  'application stay': 65653,
  'stay execution': 1000003,
  'death penalty': 280925,
  'certiorari': 157932,
  'penalty': 753114,
  'writ': 1137503,
  'writ certiorari': 1137535}}

In [442]:
copy = big_df.copy()

In [451]:
copy

,year_1,year_2,title,topic,terms,term_frequency,lsa_bow,years
0,1870,1880,Worthy v. Commissioners,3,"[authority exercise, jurisdiction, treaty, dis...","{'authority exercise': '2750', 'jurisdiction':...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
1,1870,1880,Osborn v. Nicholson,0,"[slave, jurisdiction, cease, contract, pleadin...","{'slave': '32132', 'jurisdiction': '18601', 'c...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
2,1870,1880,Bradwell v. State,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
3,1870,1880,Slaughter-House Cases,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
4,1870,1880,Bartemeyer v. Iowa,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
5,1870,1880,Minor v. Happersett,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
6,1870,1880,Scholey v. Rew,5,"[tax, tax duty, estate, succession, duty, succ...","{'tax': '33814', 'tax duty': '33842', 'estate'...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
7,1870,1880,Raymond v. Thomas,0,"[slave, jurisdiction, cease, contract, pleadin...","{'slave': '32132', 'jurisdiction': '18601', 'c...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
8,1870,1880,Walker v. Sauvinet,4,"[trial, jury, suit common, trial jury, process...","{'trial': '34970', 'jury': '18737', 'suit comm...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"
9,1870,1880,United States v. REESE,2,"[citizen, property, government, privilege, per...","{'citizen': '5000', 'property': '27082', 'gove...","{'ship vessel': '31933', 'water': '36753', 'tr...","(1870, 1880)"


In [453]:
freq = copy["term_frequency"]

In [498]:
kdataframe = pd.DataFrame.from_records(kmdict)

In [520]:
kdataframe.head(10)

,"(1870, 1880)","(1880, 1890)","(1890, 1900)","(1900, 1910)","(1910, 1920)","(1920, 1930)","(1930, 1940)","(1940, 1950)","(1950, 1960)","(1960, 1970)","(1970, 1980)","(1980, 1990)","(1990, 2000)","(2000, 2010)","(2010, 2020)"
0,"{0: {'slave': 32132, 'jurisdiction': 18601, 'c...","{0: {'injury': 38222, 'iowa': 39599, 'damage':...","{0: {'property': 127094, 'land': 91710, 'asses...","{0: {'depot': 70756, 'village': 245996, 'stati...","{0: {'tax': 226546, 'property': 182034, 'busin...","{0: {'commission': 38876, 'rate': 172836, 'cit...","{0: {'carrier': 37890, 'highway': 132409, 'veh...","{0: {'confession': 69537, 'ashcraft': 24832, '...","{0: {'trial': 274710, 'habeas': 124011, 'habea...","{0: {'search': 542864, 'arrest': 44212, 'offic...","{0: {'inmate': 546182, 'parole': 742535, 'pris...","{0: {'death': 280198, 'stay': 999911, 'executi...","{0: {'search': 654027, 'fourth amendment': 308...","{0: {'candidate': 79551, 'voter': 607598, 'pol...","{0: {'racial': 416204, 'voter': 557586, 'race'..."
1,"{1: {'passenger': 24168, 'commerce': 6094, 've...","{1: {'juror': 40788, 'jury': 40949, 'indictmen...","{1: {'railroad': 133114, 'corporation': 39519,...","{1: {'assessment': 17601, 'improvement': 11654...","{1: {'sale': 207664, 'liquor': 137159, 'sell':...","{1: {'trust': 213727, 'tax': 204902, 'trustee'...","{1: {'tax': 267406, 'property': 215727, 'trust...","{1: {'gas': 146161, 'commission': 61432, 'rate...","{1: {'tax': 265462, 'commerce': 49723, 'inters...","{1: {'apportionment': 38042, 'election': 20653...","{1: {'obscene': 708038, 'material': 647624, 'o...","{1: {'congress': 222862, 'tax': 1034136, 'acti...","{1: {'trial': 735475, 'defendant': 201855, 'ha...","{1: {'school': 511114, 'student': 546864, 'rel...","{1: {'trial': 534956, 'juror': 283056, 'defend..."
2,"{2: {'citizen': 5000, 'property': 27082, 'gove...","{2: {'congress': 15328, 'militia': 47326, 'yar...","{2: {'jury': 89505, 'verdict': 170504, 'eviden...","{2: {'power': 174652, 'contract': 55307, 'wate...","{2: {'employer': 82327, 'employee': 82147, 'ho...","{2: {'advocate': 4643, 'syndicalism': 204456, ...","{2: {'milk': 173852, 'power': 204669, 'contrac...","{2: {'jury': 185541, 'religious': 283968, 'pub...","{2: {'jury': 150797, 'confession': 56662, 'gra...","{2: {'trial': 612933, 'jury': 333411, 'counsel...","{2: {'school': 943066, 'child': 167786, 'publi...","{2: {'death': 280198, 'jury': 585792, 'death p...","{2: {'congress': 153567, 'government': 322948,...","{2: {'congress': 121567, 'government': 253978,...","{2: {'juvenile': 285439, 'parole': 364343, 'li..."
3,"{3: {'authority exercise': 2750, 'jurisdiction...","{3: {'railroad': 62070, 'tax': 73343, 'propert...","{3: {'citizen': 25268, 'power': 120990, 'comme...","{3: {'tax': 227479, 'property': 183387, 'taxat...","{3: {'land': 130494, 'water': 246470, 'canal':...","{3: {'public': 168424, 'business': 26481, 'pow...","{3: {'tax': 267406, 'city': 45247, 'business':...","{3: {'tax': 325712, 'property': 263246, 'estat...","{3: {'insurance': 142285, 'highway': 128494, '...","{3: {'confession': 124961, 'habeas': 274550, '...","{3: {'search': 948016, 'fourth amendment': 447...","{3: {'trial': 1065064, 'defendant': 291016, 'c...","{3: {'voting': 769286, 'election': 248940, 'ca...","{3: {'inmate': 296626, 'prison': 439672, 'osp'...","{3: {'cake': 74317, 'phillips': 374639, 'speec..."
4,"{4: {'trial': 34970, 'jury': 18737, 'suit comm...","{4: {'citizen': 11416, 'congress': 15328, 'cou...","{4: {'tax': 158087, 'taxation': 158911, 'bank'...","{4: {'insurance': 122522, 'policy': 172973, 'l...","{4: {'commission': 41955, 'railroad': 189513, ...","{4: {'tax': 204902, 'corporation': 52187, 'bus...","{4: {'tax': 267406, 'interstate': 146689, 'com...","{4: {'insurance': 174871, 'commerce': 60701, '...","{4: {'new': 178818, 'government': 120062, 'pub...","{4: {'school': 540403, 'congress': 127046, 'pu...","{4: {'jury': 585502, 'trial': 1067876, 'defend...","{4: {'school': 942679, 'religious': 890486, 'r...","{4: {'tax': 713438,

In [522]:
flatten_all(kdataframe[(1870, 1880)].iloc[3])["treaty"]

'34955'

In [ ]:
for column in kdataframe.columns:
    for term in flatten_all(kdataframe[column].iloc[i]).keys():
        print(term)

In [532]:
kdataframe[(1870, 1880)].iloc[0]

{0: {'slave': 32132,
  'jurisdiction': 18601,
  'cease': 4381,
  'contract': 8023,
  'pleading': 25163,
  'warranty': 36717,
  'illinois': 16658,
  'order': 23343,
  'thomas': 34487,
  'raymond': 28678}}

In [ ]:
kdataframe

In [608]:
flatten_all(kdataframe[column].iloc[i]).keys()

dict_keys(['tax', 'taxpayer', 'flast', 'winn', 'tax credit', 'religious', 'expenditure', 'tuition', 'school tuition', 'arizona'])

In [688]:
graph_df = pd.DataFrame(columns=["term","year_1","year_2","topic","frequency"])
for column in kdataframe.columns:
    for i in range(len(kdataframe)):
        terms = {"term":[],"year_1":[],"year_2":[],"topic":[],"frequency":[]}
        for key in flatten_all(kdataframe[column].iloc[i]).keys():
            terms["term"].append(key)
            terms["year_1"].append(column[0])
            terms["year_2"].append(column[1])
            terms["topic"].append(list(kdataframe[column].iloc[i].keys())[0])
            terms["frequency"].append(flatten_all(kdataframe[column].iloc[i])[key])
            
        term_df = pd.DataFrame.from_dict(terms)
        graph_df = pd.concat([graph_df, term_df], ignore_index=True)

In [704]:
graph_df.head(25)

,term,year_1,year_2,topic,frequency
0,slave,1870,1880,0,32132
1,jurisdiction,1870,1880,0,18601
2,cease,1870,1880,0,4381
3,contract,1870,1880,0,8023
4,pleading,1870,1880,0,25163
5,warranty,1870,1880,0,36717
6,illinois,1870,1880,0,16658
7,order,1870,1880,0,23343
8,thomas,1870,1880,0,34487
9,raymond,1870,1880,0,28678


In [ ]:
graph_df[graph_df.isna().any(axis=1)]

In [694]:
graph_df["frequency"] = graph_df["frequency"].apply(lambda x: int(x))

In [702]:
graph_df["year_2"].apply(lambda x: int(x))
type(graph_df["term"].iloc[0])

str

In [ ]:
graph_df_2 = pd.DataFrame(columns=["year_1", "year_2", "title", "topic","terms", "term_frequency", "lsa_bow"])

for k in topic_and_data.keys():
    for i in range(len(topic_and_data[k][0])):
        terms = {"year_1":[],"year_2":[], "title":[],"topic":[],"terms":[], "term_frequency":[], "lsa_bow":[]}
        range_df.loc[i] = {"year_1":k[0],"year_2":k[1],"title":topic_and_data[k][0][i][0][0],
                     "topic":topic_and_data[k][0][i][1],"terms":[kmb[k][topic_and_data[k][0][i][1]]],
                     "term_frequency":[kmdict[k][topic_and_data[k][0][i][1]]],"lsa_bow":[lsat[k]]}
    # big_df_2 = pd.concat([big_df, range_df], axis=0)

In [644]:
import plotly.express as px
fig = px.scatter(graph_df, x="term", y="topic", animation_frame="year_1", 
           size="frequency", color="topic", size_max=45, range_x=[1760,2050], range_y=[-1,6])
fig.show()

In [ ]:
# import spacy
# from spacy.tokens import Token
# nlp = spacy.load('en_core_web_sm')
# 
# doc = nlp(str(binned_data[(1960, 1970)][:,1][311]))
# print(set([token.text for token in doc if token.ent_type_ == "GPE"]))